In [ ]:
import os

if os.name == 'nt':
    measuring_root = "D:/measuring"
else:
    measuring_root = "/Users/jesse/Thesis/Code"

execfile(os.path.join(measuring_root, "analysis/scripts/setup_analysis.py"))
import analysis.lib.purification.purify_delayfb as pu_delayfb;reload(pu_delayfb)
import analysis.lib.fastcarboncontrol.fcc as fcc; reload(fcc)
import matplotlib as mpl
mpl.style.use('seaborn-bright')
%matplotlib inline

In [ ]:
reload(pu_delayfb)
carbons = [5,4]

single_C_decay_data = dict()

for c in carbons:
    data = dict()
    data["x"], data["y"], data["y_u"], data["fr"] = \
    pu_delayfb.number_of_repetitions(contains = 'sweep_number_of_reps_C%d_X' % (c),do_fit = True,fixed=[0,2,5,6,4], ret_data_fit=True)
    single_C_decay_data[str(c)] = data

In [ ]:
reload(pu_delayfb)
carbons = [2,4,5,3,6,7]

repump_time_data = dict()

for c in carbons:
    data = dict()
    data["x"], data["y"], data["y_u"], data["fr"] = \
    pu_delayfb.average_repump_time(contains = 'Sweep_Repump_time_C%d_X' % (c),do_fit = True,fixed=[], ret_data_fit=True)
    repump_time_data[str(c)] = data

In [ ]:
import itertools
reload(pu_delayfb)

carbons = [5,4]
carbon_combis = [(5,4)]

print("All carbon combinations: " + str(carbon_combis))

carbon_combis = carbon_combis

print("Selected carbon combinations: " + str(carbon_combis))

C13_X_phase = 0.0

msmt_sweep_limits = [
    (10, 190, 30),
    (190, 340, 30),
    (340, 850, 120),
]

msmt_templates = [
    {
        'carbon_encoding':              'serial_swap',
        'carbon_swap_el_states':        ['Z', 'Z'],
        'Tomography_list': [
            # ['X', 'I'],
            # ['I', 'X'],
            ['X', 'X'],
            [C13_X_phase + 45.0, C13_X_phase + 45.0],
            [C13_X_phase + 45.0, C13_X_phase - 45.0],
        ]
    },
    {
        'carbon_encoding':              'MBE',
        'carbon_swap_el_states':        ['X'],
        'Tomography_list': [
            ['X', 'X'],
            ['Y', 'Y'],
            ['Z', 'Z'],
            ['X', 'Y'],
            ['Y', 'X'],
        ]
    },
    {
        'carbon_encoding':              'MBE',
        'carbon_swap_el_states':        ['-X'],
        'Tomography_list': [
            ['X', 'X'],
            ['Y', 'Y'],
            ['Z', 'Z'],
            ['X', 'Y'],
            ['Y', 'X'],
        ]
    }
]

results = {}

for combi in carbon_combis:
    combi_key = "".join([str(c) for c in combi])
    results[combi_key] = {}
    for m in msmt_templates:
        dictkey = "%s_%s" % (m['carbon_encoding'], "".join(m['carbon_swap_el_states']))
        results[combi_key][dictkey] = {}
        folder_names = []
        for d_i, d in enumerate(msmt_sweep_limits):
            name_template = "phase_fb_delayline_C%s_%s_%s_sec%d" % (
                    "".join([str(c) for c in combi]),
                    m['carbon_encoding'],
                    "".join(m['carbon_swap_el_states']),
                    d_i
            )
            folder_names += [name_template]
        print folder_names
        for t in ["".join([str(tbasis) for tbasis in tbases]) for tbases in m['Tomography_list']]:
            print("Tomo: %s" % t)
            data = dict()
            data["x"], data["y"], data["y_u"], data["fr"] = \
            pu_delayfb.number_of_repetitions_stitched(
                contains="",
                multi_contains=folder_names,
                do_fit=True,
                x_only=True,
                tomo_basis=t,
                fit_n=1.0,
                fixed=[0,2,5,6,4],
                ret_data_fit=True,
                T2star_correction=(not "Z" in t),
                LDE_element_length=7e-6
            )
            results[combi_key][dictkey][t] = data
                
            
            
# import copy
# msmts_and_data = copy.copy(msmts)

# for m in msmts.iterkeys():
#     print("Msmt: "+m)
#     ots = msmts[m]["ots"]
#     msmts_and_data[m]["tomos_data"] = dict()
#     for t in msmts[m]["tomos"]:
#         print("Tomo: %s - %s" % (m, t))
#         data = dict()
#         data["x"], data["y"], data["y_u"], data["fr"] = \
#         pu_delayfb.number_of_repetitions_stitched(
#             contains="phase_fb_delayline",
#             older_thans=ots,
#             do_fit=True,
#             x_only=True,
#             tomo_basis=t,
#             fit_n=1.0,
#             fixed=[0,2,5,6,4],
#             ret_data_fit=True
#         )
#         msmts_and_data[m]["tomos_data"][t] = data

In [ ]:
plt.figure()

from analysis.lib.fitting import fit, common

xvals = np.linspace(0,1000,1001)
for combi_key in results.iterkeys():
    for m_key in results[combi_key].iterkeys():
        if 'MBE' not in m_key:
            continue
        
        m_data = results[combi_key][m_key]
        tomos = ['XX', 'YY', 'YX', 'XY']
        bloch_length = np.zeros_like(m_data[tomos[0]]['y'])
        bloch_length_u = np.zeros_like(m_data[tomos[0]]['y'])
        for t in tomos:
            bloch_length += m_data[t]['y'] ** 2
            bloch_length_u += (m_data[t]['y_u'] * m_data[t]['y']) ** 2
        bloch_length = np.sqrt(bloch_length) / np.sqrt(2)
        bloch_length_u = np.sqrt(bloch_length_u) / bloch_length / np.sqrt(2)
        
        g_a = 0
        g_A = bloch_length[0]
        g_x0 = 0
        g_T = 500
        g_n = 1
        g_f = 0.0000
        g_phi = 0
        fixed = [0,2,4,5,6]
        
        p0, fitfunc, fitfunc_str = common.fit_exp_cos(g_a, g_A, g_x0, g_T, g_n, g_f, g_phi)

        fit_result = fit.fit1d(m_data[tomos[0]]['x'], bloch_length, None, p0=p0, fitfunc=fitfunc, do_print=True, fixed=fixed, ret=True)
        
        results[combi_key][m_key]['bloch'] = dict({
            'x': m_data[tomos[0]]['x'],
            'y': bloch_length,
            'y_u': bloch_length_u,
            'fr': fit_result
        })

        
        plt.errorbar(m_data[tomos[0]]['x'], bloch_length, bloch_length_u, label=combi_key + " " + m_key, fmt='o')
        plt.plot(xvals, fit_result['fitfunc'](xvals))
        
        
        
plt.legend()

In [ ]:
interesting_msmts = [
    ("MBE_-X", "XX", r"MBE -X, $\langle XX \rangle$"), 
    ("MBE_-X", "ZZ", r"MBE -X, $\langle ZZ \rangle$"), 
    ("MBE_-X", "bloch", r"MBE -X, equatorial bloch vector length"),
    ("MBE_X", "XX", r"MBE +X, $\langle XX \rangle$"), 
    ("MBE_X", "ZZ", r"MBE +X, $\langle ZZ \rangle$"), 
    ("MBE_X", "bloch", r"MBE +X, equatorial bloch vector length"),
    ("serial_swap_ZZ", "45.045.0", r"sep. XX, $\langle X+45\degree, X+45\degree \rangle$"),
    ("serial_swap_ZZ", "45.0-45.0", r"sep. XX, $\langle X+45\degree, X-45\degree \rangle$")
]

fit_xvals = np.linspace(0.,1000., 2000.)

colors = ['b','r','g','m','y','c','k', '0.5']

for combi_key in results.iterkeys():
    fig = plt.figure()
    ax = plt.subplot()
    
    i = 0
    for i_m in interesting_msmts:
        data = results[combi_key][i_m[0]][i_m[1]]
        plt.errorbar(
            data["x"], data["y"], data["y_u"], 
            fmt='o', color=colors[i], #"C%d" % i, 
            label = r"%s, $N_{1/e} = %d \pm %d$" % (i_m[2], np.round(data["fr"]['params_dict']['T']),
                                                                     np.round(data["fr"]['error_dict']['T']))
        )
        plt.plot(fit_xvals, data["fr"]['fitfunc'](fit_xvals), color=colors[i]) #"C%d" % i)
        i += 1

    plt.ylim(-1,1)
    plt.xlim(0,1000)

    plt.xlabel("Number of LDE attempts")
    plt.ylabel(r"Expectation value")
    plt.title("Correlations for carbon combination C%s" % ", C".join(combi_key))

    lgd = plt.legend(loc=2, bbox_to_anchor=(1,1))    
    plt.savefig("batch-plots/carbon-correlations-%s.png" % combi_key, dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.show()
plt.close('all')

In [ ]:
fig = plt.figure()
ax = plt.subplot()
i=0
for c_str in single_C_decay_data.iterkeys():
    data = single_C_decay_data[c_str]
    plt.errorbar(
        data["x"], data["y"], data["y_u"], 
        fmt='o', color="C%d" % i, 
        label = r"C%s, $N_{1/e} = %d \pm %d$" % (c_str, np.round(data["fr"]['params_dict']['T']),
                                                                 np.round(data["fr"]['error_dict']['T']))
    )
    plt.plot(fit_xvals, data["fr"]['fitfunc'](fit_xvals), color="C%d" % i)
    i += 1

plt.ylim(0,1)
plt.xlim(0,600)

plt.xlabel("Number of LDE attempts")
plt.ylabel(r"Bloch vector length")
# plt.title("Single-nuclear state decay")

lgd = plt.legend(loc=2, bbox_to_anchor=(1,1))    
plt.savefig("batch-plots/single-nuclear-state-decay.png", dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()
plt.close('all')

In [ ]:
import measurement.scripts.lt4_scripts.setup.msmt_params as msmt_params
reload(msmt_params)

trans = "m1"

LDE_length = 7e-6
dec_time = 2.256e-6 * 2
ms0_time = LDE_length - dec_time

couplings = []
deviations = []

for c in carbons:
    print ("Carbon: %d" % c)
    ms0_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_0' % c]
    ms1_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_1_%s' % (c, trans)]
    avg_freq = msmt_params.cfg['samples']['111no2']['C%d_freq_%s' % (c, trans)]
    calibrated_phase_deg = msmt_params.cfg['samples']['111no2']['C%d_phase_per_LDE_sequence_%s' % (c, trans)]
    
    parallel_coupling = ms1_freq - ms0_freq
    print("parallel coupling (kHz): \t%.3f" % (parallel_coupling*1e-3))
    
    phase = ms0_time * ms0_freq + dec_time * avg_freq
    print("phase (rotations): \t\t%.3f" % phase)
    phase_deg = (phase % 1.0) * 360.0
    print("phase (degrees): \t\t%.3f" % phase_deg)
    print("phase (calibrated): \t\t%.3f" % calibrated_phase_deg)
    
    phase_dev = phase_deg - calibrated_phase_deg
    print("phase deviation: \t\t%.3f" % phase_dev)
    
    couplings.append(parallel_coupling)
    deviations.append(phase_dev)
    
couplings = np.array(couplings) * 1e-3
deviations = np.array(deviations)

plt.scatter(couplings, deviations)

phasedevfit = np.polyfit(couplings, deviations, 1)
p = np.poly1d(phasedevfit)

plt.plot(couplings, p(couplings))
plt.xlabel("parallel hyperfine coupling (kHz)")
plt.ylabel("phase deviation (degrees)")

print("fit parameters: " + str(phasedevfit))

In [ ]:
coupling_pairs = list(itertools.combinations(couplings, 2))[:11]

synthetic_coupling_pX = np.array([it[0] + it[1] for it in coupling_pairs])
synthetic_coupling_mX = np.array([it[0] - it[1] for it in coupling_pairs])

pX_decays = np.zeros((len(carbon_combis)))
pX_decays_u = np.zeros((len(carbon_combis)))
mX_decays = np.zeros((len(carbon_combis)))
mX_decays_u = np.zeros((len(carbon_combis)))

sC_decays = np.zeros((len(carbons)))
sC_decays_u = np.zeros((len(carbons)))

for i, combi in enumerate(carbon_combis):
    combi_key = "".join([str(c) for c in combi])
    pX_decays[i] = results[combi_key]['MBE_X']['bloch']['fr']['params_dict']['T']
    pX_decays_u[i] = results[combi_key]['MBE_X']['bloch']['fr']['error_dict']['T']
    mX_decays[i] = results[combi_key]['MBE_-X']['bloch']['fr']['params_dict']['T']
    mX_decays_u[i] = results[combi_key]['MBE_-X']['bloch']['fr']['error_dict']['T']
    
for i, c in enumerate(carbons):
    sC_decays[i] = single_C_decay_data[str(c)]['fr']['params_dict']['T']
    sC_decays_u[i] = single_C_decay_data[str(c)]['fr']['error_dict']['T']

In [ ]:
plt.figure()

plt.errorbar(np.abs(synthetic_coupling_pX), pX_decays, pX_decays_u, fmt='o', label="MBE +X")
plt.errorbar(np.abs(synthetic_coupling_mX), mX_decays, mX_decays_u, fmt='o', label="MBE -X")
plt.errorbar(np.abs(couplings), sC_decays, sC_decays_u, fmt='o', label="single carbons")

plt.xlabel("(Synthetic) coupling strength (kHz)")
plt.ylabel("Nuclear state 1/e decay constant (LDE attempts)")

plt.legend()
plt.ylim(0,1000)

plt.savefig("batch-plots/decay-vs-coupling.png", dpi=300)

In [ ]:
print carbon_combis[np.argmax(pX_decays)]
print carbons[np.argmax(sC_decays)]